In [1]:
import numpy as pd
import torch
import torch.nn as nn

from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection
from Evolution import Evolution

In [2]:
chromosomes_size = 3
index = [1, 0, 2]
fitness = [1, 0, 2]

In [3]:
x = torch.arange(0, 9, step=1).reshape(chromosomes_size, 3).unsqueeze(dim=0).float()
after_x = x[:, index]
# model = nn.Conv1d(3*chromosomes_size, 2*chromosomes_size, kernel_size=1, stride=1, groups=chromosomes_size)
model = torch.nn.Sequential(
    nn.Conv1d(3*chromosomes_size, 2*chromosomes_size, kernel_size=1, stride=1, groups=chromosomes_size),
    nn.Conv1d(2*chromosomes_size, 3*chromosomes_size, kernel_size=1, stride=1, groups=chromosomes_size)
                            )
model.eval()
model.requires_grad=False
init_model = deepcopy(model)
with torch.no_grad():
    init_out = model(x.reshape(1, -1, 1))

In [4]:
after_shape_each_layer = []
after_num_each_layer = []
for name, param in model.named_parameters():
    size = list(param.size())
    after_shape_each_layer.append(size)
    after_num_each_layer.append(param.numel())

In [9]:
device = next(model.parameters()).device
model.cpu()
with torch.no_grad():
    chromosomes = []
    ch = model
    for name, param in ch.named_parameters():
        param = param.flatten().reshape(chromosomes_size, -1)
        chromosomes.append(param)
    chromosomes = torch.concat(chromosomes, dim=1)
chromosomes = chromosomes[index]

In [10]:
with torch.no_grad():
    torch_chromosomes = []
    ch = model
    for name, param in ch.named_parameters():
        param = param.flatten().reshape(chromosomes_size, -1)
        torch_chromosomes.append(param)
    torch_chromosomes = torch.concat(torch_chromosomes, dim=1)
torch_chromosomes = torch_chromosomes[index]

In [12]:
sd = model.state_dict()
split_base = 0
for idx_sd, param_name in enumerate(sd):
    split_margin = split_base + after_num_each_layer[idx_sd] // chromosomes_size
    param = chromosomes[:, split_base:split_margin].reshape(after_shape_each_layer[idx_sd])
    sd[param_name] = param
    split_base = split_margin
model.load_state_dict(sd)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


<All keys matched successfully>

In [20]:
# Update after layers
sd = model.state_dict()
split_base = 0
for idx_sd, param_name in enumerate(sd):
    split_margin = split_base + after_num_each_layer[idx_sd] // chromosomes_size
    param = chromosomes[:, split_base:split_margin].reshape(after_shape_each_layer[idx_sd])
    sd[param_name] = param
    split_base = split_margin
model.load_state_dict(sd)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


<All keys matched successfully>

In [21]:
model.eval()
with torch.no_grad():
    after_out = model(after_x.reshape(1, -1, 1))

In [22]:
init_out

tensor([[[ 0.1007],
         [-0.4575],
         [ 0.3629],
         [ 1.5556],
         [-1.0292],
         [-0.9668],
         [ 2.5570],
         [ 0.0846],
         [ 0.7617]]])

In [23]:
after_out

tensor([[[ 1.5556],
         [-1.0292],
         [-0.9668],
         [ 0.1007],
         [-0.4575],
         [ 0.3629],
         [ 2.5570],
         [ 0.0846],
         [ 0.7617]]])